#### 분해하려는 원본 행렬 R을 P와 Q로 분해한 후에 다시 P와 t(Q)의 내적으로 예측 행렬의 만드는 예제 

In [1]:
import numpy as np

In [4]:
#원본 행렬 R생성, 분해행렬 P와 Q초기화, 잠재 요인 차원 K는 3으로 설정
R= np.array([[4,np.NaN, np.NaN,2 ,np.NaN],[np.NaN,5,np.NaN,3,1],[np.NaN,np.NaN,3,4,4],[5,2,1,2,np.NaN]])

num_users, num_items=R.shape
K=3

In [6]:
# P와 Q행렬의 크기를 지정하고 정규분포를 가진 임의의 값으로 입력
np.random.seed(1)
P=np.random.normal(scale=1./K, size=(num_users,K))
Q=np.random.normal(scale=1./K, size=(num_items,K))

In [8]:
# 실제 R행렬과 예측 행렬의 오차를 구하는 get_rmse()함수

from sklearn.metrics import mean_squared_error

In [13]:
def get_rmse(R,P,Q,non_zeros):
    error=0
    #두개의 분해된 행렬 P와 T(Q)의 내적으로 예측 행렬 생성
    full_pred_matrix= np.dot(P,Q.T)
    
    #실제 R행렬에서 널이 아닌 값의 위치 인덱스 추출해 실제 R행렬과 예측 행렬의 RMSE값 추출
    x_non_zero_ind= [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind= [non_zero[1] for non_zero in non_zeros]
    R_non_zeros=R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros=full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    mse=mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse=np.sqrt(mse)
    
    return rmse

In [14]:
# R>0인 행의 위치, 열의 위치, 값을 non-zeros 리스트에 저장
non_zeros = [(i,j,R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j]>0]

steps=1000
learning_rate=0.01
r_lambda=0.01

#SGD 기법으로 P와 Q매트릭스를 계속 업데이트
for step in range(steps):
    for i,j,r in non_zeros:
        #실제 값과 예측값의 차이인 오류값 구함
        eij=r-np.dot(P[i,:],Q[j,:].T)
        # 규제를 반영한 SGD업데이트 공식 적용
        P[i,:]=P[i,:]+learning_rate*(eij*Q[j,:]-r_lambda*P[i,:])
        Q[j,:]=Q[j,:]+learning_rate*(eij*P[i,:]-r_lambda*Q[j,:])
        rmse=get_rmse(R,P,Q,non_zeros)
        if (step%50)==0:
            print('### iteration step: ', step, "rmse: ",rmse)

### iteration step:  0 rmse:  3.257685974919492
### iteration step:  0 rmse:  3.2567784112905804
### iteration step:  0 rmse:  3.2503556288140967
### iteration step:  0 rmse:  3.248564885598423
### iteration step:  0 rmse:  3.2487420072644557
### iteration step:  0 rmse:  3.2474782606248875
### iteration step:  0 rmse:  3.2456461223335706
### iteration step:  0 rmse:  3.2438156312541464
### iteration step:  0 rmse:  3.2385009318957363
### iteration step:  0 rmse:  3.2366485436771972
### iteration step:  0 rmse:  3.2367673519475613
### iteration step:  0 rmse:  3.2353964232171952
### iteration step:  50 rmse:  0.5021726731206282
### iteration step:  50 rmse:  0.501958220155996
### iteration step:  50 rmse:  0.5008855357324867
### iteration step:  50 rmse:  0.5006882396215355
### iteration step:  50 rmse:  0.5007896401798277
### iteration step:  50 rmse:  0.500138053484773
### iteration step:  50 rmse:  0.5002489493156104
### iteration step:  50 rmse:  0.499774261611162
### iteration ste

In [15]:
pred_matrix =np.dot(P,Q.T)
print('예측행렬:\n', np.round(pred_matrix,3))

예측행렬:
 [[3.99  0.669 1.378 2.003 1.794]
 [6.716 4.978 0.929 2.98  1.003]
 [7.173 0.45  2.987 3.976 3.986]
 [4.967 2.005 1.008 2.02  1.184]]
